<a href="https://colab.research.google.com/github/RyuJaeJeong/llm-study-repo/blob/master/6_sLLM_%ED%95%99%EC%8A%B5%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 필요 라이브러리 설치

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip uninstall torch torchvision torchaudio triton bitsandbytes transformers -y
!pip install torch==2.5.1 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install triton==3.0.0
!pip install bitsandbytes==0.44.1
!pip install transformers==4.46.0

In [ ]:
!pip install huggingface_hub autotrain-advanced -qqq

# SQL 생성 프롬프트

In [ ]:
def make_prompt(ddl, question, query=''):
  prompt = f"""당신은 SQL을 생성하는 SQL봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.
  ### DDL:
  {ddl}

  ### Question:
  {question}

  ### SQL:
  {question}"""
  return prompt

# GPT 평가 프롬프트와 코드 준비

In [ ]:
from pathlib import Path
import json

def make_requests_for_gpt_evaluation(df, filename, dir='requests'):
  if not Path(dir).exists():
    Path(dir).mkdir(parents=True)
  prompts=[]
  for idx, row in df.iterrows():
    prompts.append("""Based on below DDL and Question, evaluate gen_sql can resolve Question. If gen_sql and gt_sql do equal job, return \"yes\" else return \"no\". Output JSON Format: {\"resolve_yn\": \"\"}""" + f"""
      DDL: {row['context']}
      Question: {row['question']}
      gen_sql: {row['gen_sql']}
      gt_sql: {row['answer']}"""
    )
  jobs = [{"model": "gpt-4o", "response_format": {"type": "json_object"}, "messages": [{"role": "system", "content": prompt}]} for prompt in prompts]
  with open(Path(dir, filename), "w", encoding="utf-8") as f:
    for job in jobs:
      json_string = json.dumps(job, ensure_ascii=False)
      f.write(json_string + '\n')

# 기초 모델로 생성하기

In [ ]:
from huggingface_hub import snapshot_download
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

# 원하는 저장 경로 설정 (Google Drive 내부 폴더)
local_dir = "/content/drive/MyDrive/models/Yi-Ko-6B"

# 모델 다운로드 및 저장
snapshot_download(repo_id="beomi/Yi-Ko-6B", local_dir=local_dir)

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

def make_inference_pipeline(model_id):
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  # model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', torch_dtype=torch.bfloat16)
  quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16
  )
  model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', quantization_config=quant_config)
  pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
  return pipe

model_id = "/content/drive/MyDrive/models/Yi-Ko-6B"
hf_pipe = make_inference_pipeline(model_id)
example = """당신은 SQL을 생성하는 SQL봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
CREATE TABLE users (
  player_id INT PRIMARY KEY AUTO_INCREMENT,
  username VARCHAR(255) UNIQUE NOT NULL,
  email VARCHAR(255) UNIQUE NOT NULL
  password_hash VARCHAR(255) NOT NULL,
  date_joined DATETIME NOT NULL,
  last_login DATETIME
);

### Question:
사용자 이름에 'admin'이 포함되어 있는 계정의 수를 알려주세요

### SQL:
"""
hf_pipe(example, do_sample=False, return_full_text=False, max_length=1024, truncation=True)

# 기초 모델 성능 측정

In [ ]:
from datasets import load_dataset

# 데이터셋 불러오기
df = load_dataset("shangrilar/ko_text2sql", "origin")['test']
df = df.to_pandas()
for idx, row in df.iterrows():
  prompt = make_prompt(row['context'], row['question'])
  df.loc[idx, 'prompt'] = prompt

# sql 생성
gen_sqls = hf_pipe(df['prompt'].tolist(), do_sample=False, return_full_text=False, max_length=512, truncation=True)
gen_sqls = [x[0]['generated_text'] for x in gen_sqls]
df['gen_sql'] = gen_sqls

# 평가를 위한 requests.jsonl 생성
eval_filepath = "text2sql_evaluation.jsonl"
make_requests_for_gpt_evaluation(df, eval_filepath)

In [ ]:
# GPT-4 평가수행
!python api_request_parallel_processor.py \
  --requests_filepath requests/{eval_filepath} \
  --save_filepath requests/{eval_filepath} \
  --request_url https://api.openai.com/v1/chat/completions \
  --max_requests_per_minute 2500 \
  --token_encoding_name cl100k_base \
  --max_attempts 5 \
  --logging_level 20